# Introduction
Memote shows us a list of orphan metabolites: that are consumed by not produced. There are only 37 metabolites that fall in this category. Here I will go through this list and re-evaluate if and how they can be connected to central metabolism. In principle having some orphan metabolites is not a problem, as they can come from knowledge gaps. But, they do contribute to blocked reactions. Ideally we should get no more than 5-10% blocked reactions. So checking and alleviating some of the orphan metabolites may also help improve the % of blocked reactions. 

A second way to improve % of blocked reactions is by looking at the dead-end metabolites and evaluating if the end of the pathways should be added into biomass or if transport out of the cell should be allowed. But this is something we will look into later in another notebook, when the lipid biosynthesis is fixed and we have mass balance.

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Metabolite, Reaction

In [46]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [4]:
model_e_coli = cameo.load_model('iML1515')

In [5]:
model_b_sub = cameo.load_model('iYO844')

In [47]:
#pyridoxal is a unconnected met in the master branch
model.remove_metabolites(model.metabolites.pyridoxal_c)

In [48]:
#butanoate: add transport out of the cell
model.reactions.BUTPT.id = 'BUTKr'

In [49]:
model.reactions.BUTKr.bounds = (-1000,1000)

In [50]:
#wrong direction
model.reactions.ACOAD1f.bounds = (-1000,0)

In [51]:
model.reactions.ACACT2.bounds = (-1000,1000)

In [52]:
model.reactions.BTCOAOR.bounds = (-1000,1000)

In [53]:
#add  transport & exchange of but_c as proton symport
model.add_reaction(Reaction(id='BUTt', name = 'Transport of butanoate'))

In [54]:
model.reactions.BUTt.annotation['sbo'] = 'SBO:0000185'
model.reactions.BUTt.annotation['metanetx.reaction'] = 'MNXR96353'
model.reactions.BUTt.bounds = (-1000,1000)

In [55]:
model.groups.get_by_id('02000 - Passive transport').add_members(model.reactions.BUTt)

In [56]:
#add but_e met
model.add_metabolites(Metabolite(id='but_e',name = 'Butanoate', charge = 0, compartment = 'e'))

In [63]:
model.metabolites.but_e.formula = 'C4H8O2'
model.metabolites.but_e.annotation = model.metabolites.but_c.annotation

In [58]:
model.reactions.BUTt.add_metabolites({
    model.metabolites.but_e:-1,
    model.metabolites.but_c:1,
    model.metabolites.h_e: -1,
    model.metabolites.h_c:1
})

In [59]:
#add exchange
model.add_boundary(model.metabolites.but_e,type = 'exchange', reaction_id = 'EX_but_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_but_e
Name,Butanoate exchange
Memory address,0x0202c67150c8
Stoichiometry,but_e --> Butanoate -->
GPR,
Lower bound,0
Upper bound,1000


In [60]:
model.groups.get_by_id('00000 - Exchange reactions').add_members(model.reactions.EX_but_e)

In [64]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

__Cyanide metabolism__
Various bacilli have been shown to be able to biodegrade cyanide in the environment. Our organism also has quite some of these genes annotated. However, there is no cyanide uptake and the thiocyanate produced cannot be further converted in the current model.  It is unclear if this is really present in our specific strain, so this is a knowlede gap. Also it is not really important in general metabolism as long as cyanide is not supplied, and so this will be left as it is. 

In [35]:
#MERCPPYROR in the wrong direction
model.reactions.MERCPPYROR.bounds = (-1000,0)

mercplac is the end product of a pathway. As we don't expect it to be used in biomass it should be transported out


In [36]:
#add mercplac_e met
model.add_metabolites(Metabolite(id='mercplac_e',name = '3-Mercaptolactate', charge = 0, compartment = 'e'))

In [37]:
model.metabolites.mercplac_e.formula = 'C3H6O3S'
model.metabolites.mercplac_e.annotation = model.metabolites.mercplac_c.annotation

In [38]:
#add  transport & exchange of mercplac_e as proton symport (similar to normal lactate transport)
model.add_reaction(Reaction(id='MERCPLACt', name = 'Transport of 3-Mercaptolactate'))

In [39]:
model.reactions.MERCPLACt.annotation['sbo'] = 'SBO:0000185'
model.reactions.MERCPLACt.bounds = (-1000,1000)

In [40]:
model.groups.get_by_id('02000 - Passive transport').add_members(model.reactions.MERCPLACt)

In [41]:
model.reactions.MERCPLACt.add_metabolites({
    model.metabolites.mercplac_c:1,
    model.metabolites.mercplac_e:-1,
    model.metabolites.h_c: 1,
    model.metabolites.h_e:-1
})

In [42]:
#add exchange
model.add_boundary(model.metabolites.mercplac_e,type = 'exchange', reaction_id = 'EX_mercplac_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_mercplac_e
Name,3-Mercaptolactate exchange
Memory address,0x0202c59a5cc8
Stoichiometry,mercplac_e --> 3-Mercaptolactate -->
GPR,
Lower bound,0
Upper bound,1000


In [43]:
model.groups.get_by_id('00000 - Exchange reactions').add_members(model.reactions.EX_mercplac_e)

In [45]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

acmum: this is a kind of modified sugar that can be taken up by our strain via PTA (has the PTS transporter annotated). Therefore, to captivate this, I will remove the ACMUMPT reaction and add a transport reaction. To do so I will need to make acmum extracellular and also add the exchange that would allow one to supply it to the medium if necessary.

In [65]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [66]:
model.metabolites.acmum_c.id = 'acmum_e'

In [67]:
model.metabolites.acmum_e.compartment = 'e'

In [68]:
model.remove_reactions(model.reactions.ACMUMPT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [69]:
model.add_reaction(Reaction(id='ACMUMtpts', name = 'Tansport of acmum via PTS'))

In [70]:
model.groups.get_by_id('02060 - Phosphotransferase system (PTS)').add_members(model.reactions.ACMUMtpts)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:93: UserWarning:

need to pass in a list



In [71]:
model.reactions.ACMUMtpts.annotation['sbo'] = 'SBO:0000655'

In [72]:
model.reactions.ACMUMtpts.add_metabolites({
    model.metabolites.acmum_e:-1,
    model.metabolites.acmum6p_c:1,
    model.metabolites.pep_c:-1,
    model.metabolites.pyr_c:1,
    model.metabolites.h_c: 1
})

In [73]:
#add exchange
model.add_boundary(model.metabolites.acmum_e,type = 'exchange', reaction_id = 'EX_acmum_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_acmum_e
Name,N-Acetylmuramate exchange
Memory address,0x0202ced59288
Stoichiometry,acmum_e --> N-Acetylmuramate -->
GPR,
Lower bound,0
Upper bound,1000


In [74]:
model.groups.get_by_id('00000 - Exchange reactions').add_members(model.reactions.EX_acmum_e)

In [75]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Cellulose: need to add cellulose exchange rct, as this prevents allowing modification to grow the model on cellulose. I will put it in one direction for now, so that it is not incorporated into the default medium.

Also, i will modify the BGLUCH and CELLOHYDRO reactions to take place outside of the cell. Then the monomers that are formed are taken up according to the system they already had present. 
For this I need to modify the cellulose metabolite to be extracellular.

In [76]:
model.metabolites.cellulose_c.id = 'cellulose_e'

In [77]:
model.metabolites.cellulose_e.compartment = 'e'

In [78]:
model.reactions.BGLUCH.add_metabolites({
    model.metabolites.h2o_c:2,
    model.metabolites.glc__D_c: -2,
    model.metabolites.h2o_e:-2,
    model.metabolites.glc__D_e:2
})

In [79]:
model.reactions.CELLOHYDRO.add_metabolites({
    model.metabolites.h2o_c:1,
    model.metabolites.cellb_c:-1,
    model.metabolites.h2o_e:-1,
    model.metabolites.cellb_e:1
})

In [80]:
#add exchange
model.add_boundary(model.metabolites.cellulose_e,type = 'exchange', reaction_id = 'EX_cellulose_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_cellulose_e
Name,cellulose exchange
Memory address,0x0202ce9dc248
Stoichiometry,cellulose_e --> cellulose -->
GPR,
Lower bound,0
Upper bound,1000


In [81]:
model.groups.get_by_id('00000 - Exchange reactions').add_members(model.reactions.EX_cellulose_e)

In [82]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Selenite (slnt_c): this can be made from selenate, as the protein for that conversion seems to be present, after doing a Blast. Selenate, if used, is taken up from the environment. So here I will add transport and exchange again.

In [83]:
#add selenate in both _c and _e
model.add_metabolites(Metabolite(id='sel_c', name = 'Selenate', compartment = 'c'))

In [84]:
model.metabolites.sel_c.charge = -2
model.metabolites.sel_c.formula = 'O4Se'
model.metabolites.sel_c.annotation = model_e_coli.metabolites.sel_c.annotation

In [85]:
model.add_metabolites(Metabolite(id='sel_e', name = 'Selenate', compartment = 'e'))

In [86]:
model.metabolites.sel_e.charge = -2
model.metabolites.sel_e.formula = 'O4Se'
model.metabolites.sel_e.annotation = model_e_coli.metabolites.sel_e.annotation

In [87]:
#add conversion to selenate
model.add_reaction(Reaction(id='SELR', name = 'Selenate reductase'))

In [88]:
model.reactions.SELR.annotation = model_e_coli.reactions.SELR.annotation

In [89]:
model.reactions.SELR.add_metabolites({
    model.metabolites.sel_c: -1,
    model.metabolites.qh2_c:-1, 
    model.metabolites.h2o_c: 1,
    model.metabolites.slnt_c: 1,
    model.metabolites.ubiquin_c:1
})

In [90]:
model.metabolites.slnt_c.charge =-2

In [91]:
#in most organisms selenate is known to be transported with an ABC permease, and so I will add an atp coupled transporter here. 

In [92]:
#add sel_c transport
model.add_reaction(Reaction(id='SELtabc', name = 'Transport of Selenate via ABC transport'))

In [93]:
model.groups.get_by_id('02010 - ABC transporters').add_members(model.reactions.SELtabc)

In [94]:
model.reactions.SELtabc.annotation['sbo'] = 'SBO:0000185'

In [95]:
model.reactions.SELtabc.add_metabolites({
    model.metabolites.sel_e:-1,
    model.metabolites.atp_c: -1,
    model.metabolites.h2o_c: -1,
    model.metabolites.adp_c: 1,
    model.metabolites.h_c:1,
    model.metabolites.pi_c: 1,
    model.metabolites.sel_c:1
})

In [96]:
#add exchange
model.add_boundary(model.metabolites.sel_e,type = 'exchange', reaction_id = 'EX_sel_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_sel_e
Name,Selenate exchange
Memory address,0x0202ce9a4788
Stoichiometry,sel_e --> Selenate -->
GPR,
Lower bound,0
Upper bound,1000


In [97]:
model.groups.get_by_id('00000 - Exchange reactions').add_members(model.reactions.EX_sel_e)

In [98]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

xylu__L_c: some organisms convert L-xylulose into xylitol, with an L-xylulose reductase. But in bacilli this has not been identified yet. So we will leave this part, as it is just a knowledge gap for now.

In [99]:
#just a wrong ID was spotted
model.metabolites.xu1p__D_c.id = 'xu1p__L_c'

appl_c: Plays a role in vitamin B12 synthesis. This we will possibly revisit later anyway, so for now I will just leave it.

cbi (cobinamide): is also involved in the same pathway as appl_c so  I will leave it for now.

epm_c: This is epimelibiose, and it can be degraded but not used as a substrate. So here I will add the transport and exchange. It is generally similar to melibiose, so I will add a proton symport transport here too.

In [100]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [101]:
model.add_metabolites(Metabolite(id='epm_e', name = 'Epimelibiose', compartment = 'e', formula = 'C12H22O11', charge = 0))

In [102]:
model.metabolites.epm_e.annotation = model.metabolites.epm_c.annotation

In [103]:
model.add_reaction(Reaction(id='EPMt', name ='Transport of Epimellibiose via proton symport'))

In [104]:
model.groups.get_by_id('02000 - Passive transport').add_members(model.reactions.EPMt)

In [105]:
model.reactions.EPMt.annotation['sbo'] = 'SBO:0000185'
model.reactions.EPMt.bounds = (-1000,1000)

In [106]:
model.reactions.EPMt.add_metabolites({
    model.metabolites.epm_e:-1, 
    model.metabolites.epm_c: 1,
    model.metabolites.h_e:-1,
    model.metabolites.h_c:1
})

In [107]:
#add demand
model.add_boundary(model.metabolites.epm_e,type = 'exchange', reaction_id = 'EX_epm_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_epm_e
Name,Epimelibiose exchange
Memory address,0x0202d0522148
Stoichiometry,epm_e --> Epimelibiose -->
GPR,
Lower bound,0
Upper bound,1000


In [108]:
model.groups.get_by_id('00000 - Exchange reactions').add_members(model.reactions.EX_epm_e)

In [109]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

methylseac_c: This is part of the selenocompound metabolism pathway. Here selenocysteine is converted into selenomethionine and further into methyl-selenol. Methyl-selenic-acid (methylseac_c) can also be converted into methyl-selenol. All of these enzymes are present. 

So: 
- I will check that all of the enzymes of this pathway are properly in the model. 
- Then I will add export and demand of methyl-selenol. 
- Then also add the import & demand of methylseac, as from a physiological point of view this would be correct.

In [87]:
#hydrolysis so makes sense to be irreversible
model.reactions.SELMELY.bounds = (0,1000)

In [88]:
#Add methylselenol extracellular
model.add_metabolites(Metabolite(id='mesel_e', name = 'Methaneselenol', compartment = 'e', formula = 'CH4Se', charge =0))

In [89]:
model.metabolites.mesel_e.annotation = model.metabolites.mesel_c.annotation

In [90]:
#add transport rct
model.add_reaction(Reaction(id='MESELt', name = 'Transport of Methaneselenol', lower_bound = -1000, upper_bound = 1000))

In [91]:
model.reactions.MESELt.annotation['sbo'] = 'SBO:0000185'

In [92]:
model.groups.get_by_id('02000 - Passive transport').add_members(model.reactions.MESELt)

In [93]:
model.reactions.MESELt.add_metabolites({
    model.metabolites.mesel_e:-1,
    model.metabolites.h_e:-1,
    model.metabolites.mesel_c:1,
    model.metabolites.h_c:1
})

In [94]:
#add demand
model.add_boundary(model.metabolites.mesel_e,type = 'exchange', reaction_id = 'EX_mesel_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_mesel_e
Name,Methaneselenol exchange
Memory address,0x01f8cc7eb408
Stoichiometry,mesel_e --> Methaneselenol -->
GPR,
Lower bound,0
Upper bound,1000


In [95]:
model.groups.get_by_id('00000 - Exchange reactions').add_members(model.reactions.EX_mesel_e)

In [96]:
#add extracellular methylseac
model.add_metabolites(Metabolite(id='methylseac_e', name = 'Methylselenic acid', compartment = 'e', formula = 'CH4O2Se', charge =0 ))

In [97]:
model.metabolites.methylseac_e.annotation = model.metabolites.methylseac_c.annotation

In [98]:
#methylseac import and exchange
model.add_reaction(Reaction(id='METHYLSEACt', name = 'Transport of Methylselenic acid',lower_bound = -1000, upper_bound = 1000))

In [99]:
model.reactions.METHYLSEACt.annotation['sbo'] = 'SBO:0000185'

In [100]:
model.groups.get_by_id('02000 - Passive transport').add_members(model.reactions.METHYLSEACt)

In [101]:
#add passive, proton symport as this is an acid it is to be expected
model.reactions.METHYLSEACt.add_metabolites({
    model.metabolites.methylseac_e:-1,
    model.metabolites.h_e:-1,
    model.metabolites.methylseac_c:1,
    model.metabolites.h_c:1
})

In [102]:
#add demand
#fixed as export for now to prevent it being in the default medium
model.add_boundary(model.metabolites.methylseac_e,type = 'exchange', reaction_id = 'EX_methylseac_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_methylseac_e
Name,Methylselenic acid exchange
Memory address,0x01f8cc805108
Stoichiometry,methylseac_e --> Methylselenic acid -->
GPR,
Lower bound,0
Upper bound,1000


In [103]:
model.groups.get_by_id('00000 - Exchange reactions').add_members(model.reactions.EX_methylseac_e)

In [104]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

mnl_c: mannitol. Normally, this is taken up through the PTS system and phosphorylated in doing so. Therefore there is not really a production of mannitol anywhere, unless at some point some complex sugar can be broken down into this. Therefore I will just leave this.

isomaltose: is similar to maltose in that it has two glucose subunits. Maltose is taken up in an ABC transporter, so we could expect the same for isomaltose (as just the bond between the glucose subunits is different). I will add the transport and exchange here.

In [105]:
#add isomalt_e
model.add_metabolites(Metabolite(id='isomal_e', name = 'Isomaltose', compartment = 'e', formula = 'C12H22O11', charge = 0))

In [106]:
model.metabolites.isomal_e.annotation = model.metabolites.isomal_c.annotation

In [107]:
#add transport rct
model.add_reaction(Reaction(id='ISOMALtabc', name = 'Transport of Isomaltose via ABC transports',lower_bound = 0, upper_bound = 1000))

In [108]:
model.groups.get_by_id('02010 - ABC transporters').add_members(model.reactions.ISOMALtabc)

In [109]:
model.reactions.ISOMALtabc.annotation['sbo'] = 'SBO:0000185'

In [110]:
model.reactions.ISOMALtabc.add_metabolites({
    model.metabolites.isomal_e:-1, 
    model.metabolites.atp_c:-1,
    model.metabolites.isomal_c:1,
    model.metabolites.h2o_c:-1,
    model.metabolites.adp_c:1,
    model.metabolites.pi_c:1,
    model.metabolites.h_c:1
})

In [111]:
#add demand
#fixed as export for now to prevent it being in the default medium
model.add_boundary(model.metabolites.isomal_e,type = 'exchange', reaction_id = 'EX_isomal_e', lb = 0, ub = 1000, sbo_term = 'SBO:0000627')

Reaction identifier,EX_isomal_e
Name,Isomaltose exchange
Memory address,0x01f8cc7a7988
Stoichiometry,isomal_e --> Isomaltose -->
GPR,
Lower bound,0
Upper bound,1000


In [112]:
model.groups.get_by_id('00000 - Exchange reactions').add_members(model.reactions.EX_isomal_e)

In [113]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

tol_c: Toluene degradation is not something really observed in our strain. We have a small part of the pathway now encoded in the model. I will leave it for now, as we don't know its function. It is unconnected anyway, so doesnt matter so much. It can always be connected when more knowledge is available about this. 

tre_c: similar to mnl. Trehalose is normally taken up through PTS converting it into tre6p. So nothing really happens with the tre_c metabolite here, which is oke. If a more complex metabolite becomes added that can be hydrolyzed intracellularly to trehalose, this reaction becomes active again.

sucr_c: same as for tre_c and mnl, as sucr_e is transported with the PTS system.

gam_C: same as the other sugar. it is taken up by PTS, so only gam6p is really used. I will leave it incase some other sugar substrates are ever added that result in this.

arbt_c: same as other sugars. taken up with PTS normally and so this is for now a dead end. will eave.

Salcn_c: ditto.

acetone: it is not a product naturall made or consumed by our strain. Also the only reaction it is associated to seems wrong. so here I will remove the reaction but leave the metabolite for now.

In [114]:
model.remove_reactions(model.reactions.ACTCO2L)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list



In [115]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Catechol: 
geobacillus species seem to be able to degrade catechol, and all genes needed to do so are present in our strain. I've checked and the degradation pathway seems correctly present. However, there is no reason to believe that catechol is taken up from the environment. there is also no indication of what our strain would degrade into catechol, so I will leave this. 


2hbut_c: involved in lipid biosynthesis, so I will ignore it for now as this will be revisited later anyway.

choline: the annotation here is wrong, as well as the ID. Here we are missing reactions that both form and consume the product of the reaction this is involved in. Choline is involved in the glycerophospholipid metabolism. We will revisit lipis metabolism anyway so I will leave that for now. Also the choline is converted into betaine aldehyde, and this into betaine. The second step conversion is missing (R02565 EC:1.2.1.8), but there is no real indication what happens with betaine. It seems to function as an osmolyte,so one could expect its presence in biomass or it is exported. I will have to check this with the other first. 

So for now, I will just add the convertion of betaine aldehyde into betaine as we have genetic evidence it should be included.

In [116]:
model.metabolites.choline_c.id = 'chol_c'

In [117]:
model.metabolites.chol_c.annotation = model_e_coli.metabolites.chol_c.annotation

In [118]:
model.metabolites.betald_c.name = 'Betaine aldehyde'

In [119]:
#add the betaine met
model.add_metabolites(Metabolite(id='glyb_c', name = 'Glycine betaine', compartment = 'c' , formula = 'C5H11NO2' , charge = 0))

In [120]:
model.metabolites.glyb_c.annotation = model_e_coli.metabolites.glyb_c.annotation

In [121]:
#remove BETALDOR as you have duplicate co-factors and NADH seems correct
model.remove_reactions(model.reactions.BETALDOR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [122]:
#add reaction with NADH
model.add_reaction(Reaction(id='BETALDHx', name = 'Betaine-aldehyde dehydrogenase'))

In [123]:
model.reactions.BETALDHx.annotation = model_e_coli.reactions.BETALDHx.annotation

In [124]:
model.reactions.BETALDHx.add_metabolites({
    model.metabolites.betald_c:-1,
    model.metabolites.h2o_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.glyb_c:1,
    model.metabolites.h_c:2,
    model.metabolites.nadh_c:1
})

In [125]:
model.groups.get_by_id('00260 - Glycine, serine and threonine metabolism').add_members(model.reactions.BETALDHx)

In [126]:
#add reaction with NADPH
model.add_reaction(Reaction(id='BETALDHy', name = 'Betaine-aldehyde dehydrogenase'))

In [127]:
model.reactions.BETALDHy.annotation = model_e_coli.reactions.BETALDHy.annotation

In [128]:
model.reactions.BETALDHx.add_metabolites({
    model.metabolites.betald_c:-1,
    model.metabolites.h2o_c:-1,
    model.metabolites.nadp_c:-1,
    model.metabolites.glyb_c:1,
    model.metabolites.h_c:2,
    model.metabolites.nadph_c:1
})

In [129]:
model.groups.get_by_id('00260 - Glycine, serine and threonine metabolism').add_members(model.reactions.BETALDHy)

In [130]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

3omrsACP_c: involved in fatty acid biosynthesis, which will be re-evaluated so this can be ignored for now.

phser_c: this metabolite only has one reaction in kegg, the one we have it associated to in the model.but because this metabolite cannot be made, this whole pathway will always be blocked. So this needs further investigation. the phser_c metabolite encodes O-phosphorylhomoserine, but we also have a O-Phospho-L-homoserine metabolite present. Looking into this, it shows they are the exact same. (the phom_c can be made from phosphorylating hom__L_c as we would expect). So changing the PHSERT reaction and removing the phser_c metabolit should solve this problem.

In [131]:
model=cobra.io.read_sbml_model('../model/g-thermo.xml')

In [132]:
model.reactions.PHSERT.add_metabolites({
    model.metabolites.phser_c:1,
    model.metabolites.phom_c:-1,
    model.metabolites.h_c:-1
})

In [133]:
#remove phser_c met
model.remove_metabolites(model.metabolites.phser_c)

In [134]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

adprib_c: Here we miss a reaction that allows adp-ribose production. Looking into BRENDA; B. subtilis has an NAD+ glycohydrolase (EC 3.2.2.5) that converts NAD into adp-ribose (https://doi.org/10.1016/0076-6879(80)66451-9). There is a significant hit gound in the geobacillus genome, so I will add this reaction here. 

In [135]:
model.add_reaction(Reaction(id='NADN', name = 'NAD nucleosidase'))

In [136]:
model.reactions.NADN.annotation = model_e_coli.reactions.NADN.annotation

In [137]:
model.groups.get_by_id('00760 - Nicotinate and nicotinamide metabolism').add_members(model.reactions.NADN)

In [138]:
model.reactions.NADN.add_metabolites({
    model.metabolites.h2o_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.adprib_c:1,
    model.metabolites.h_c:1,
    model.metabolites.ncam_c:1
})

In [139]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

dmbzid_c: this can be made from the fmn2. a bacilli (Bacillus thiaminolyticus) has a confirmed BluB protein that can perform this conversion (doi: 10.1371/journal.pone.0055708). A blast of this again our strain genome gives a significant hit and so it would be reasonable to add this reaction here.
One problem is that no one knows what the exact products of this reaction is. Therefore, I will not add an incorrect reaction and so here I will do nothing.

mthpglu: a metabolite with only two reactions associated to it in kegg. So I will just leave it as we don't really know what this functions as. 

In [140]:
model=cobra.io.read_sbml_model('../model/g-thermo.xml')

In [141]:
model.metabolites.mthpglu_c.id = 'mhpglu_c'

focytB561: This is only involved in one reaction. If i look at Brenda for the anaerobic carbon-monoxide dehydrogenase reaction, it involved ferridoxin instead of ferricytochrome. Therefore I will change this reaction, making it correct and then in doing so also remove the focytB561_c and fcytb5612_c metabolites.

In [142]:
model.reactions.COOR.name = 'anaerobic carbon-monoxide dehydrogenase'

In [143]:
model.reactions.COOR.add_metabolites({
    model.metabolites.focytB561_c:2,
    model.metabolites.fcytb5612_c:-2,
    model.metabolites.fdxox_c:-1, 
    model.metabolites.fdxrd_c:1
})

In [144]:
model.remove_metabolites(model.metabolites.fcytb5612_c)

In [145]:
model.remove_metabolites(model.metabolites.focytB561_c)

In [146]:
#commit&save
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

uaaagglaa_c: plays a role in peptidoglycan biosynthesis. But as this is not really going anywhere in our model right now, so I will just leave it.

In [148]:
solution = model.optimize()

In [149]:
for rct in model.exchanges:
    flux = solution[rct.id]
    print (rct.id,' : ', flux)

EX_cl_e  :  0.0
EX_nh4_e  :  1000.0
EX_glc__D_e  :  -10.0
EX_thm_e  :  -0.004368523850940792
EX_fe2_e  :  -0.666112555971032
EX_h2o_e  :  -1000.0
EX_o2_e  :  -580.993575100424
EX_co2_e  :  969.9184045028275
EX_h_e  :  199.26083020079756
EX_cobalt2_e  :  0.0
EX_rib__D_e  :  0.0
EX_glyc_e  :  0.0
EX_asn__L_e  :  0.0
EX_pydx5p_e  :  0.0
EX_qh2_e  :  -0.005876937916013958
EX_gthrd_e  :  -396.0130106545433
EX_nadp_e  :  -0.1508414065121455
EX_Biomass_e  :  19.589793053545975
EX_gal_e  :  0.0
EX_etoh_e  :  0.0
EX_man_e  :  0.0
EX_arbt_e  :  0.0
EX_salcn_e  :  0.0
EX_tre_e  :  0.0
EX_ac_e  :  591.7300487168175
EX_succ_e  :  0.0
EX_lac__D_e  :  0.0
EX_lac__L_e  :  0.0
EX_thmtp_e  :  0.0
EX_fru_e  :  0.0
EX_arab__L_e  :  0.0
EX_xyl__D_e  :  0.0
EX_mnl_e  :  0.0
EX_cellb_e  :  0.0
EX_sucr_e  :  0.0
EX_sbt__D_e  :  0.0
EX_acgam_e  :  0.0
EX_malt_e  :  0.0
EX_rmn_e  :  0.0
EX_melib_e  :  0.0
EX_gtbi_e  :  0.0
EX_tura_e  :  0.0
EX_kdg2_e  :  0.0
EX_dglcn5_e  :  0.0
EX_mdgp_e  :  0.0
EX_tag__D_e  : 